# jupyter is so silly

In [51]:
# imports
import nltk
import glob
import os
from nltk.collocations import TrigramCollocationFinder, TrigramAssocMeasures
from nltk.corpus import EuroparlCorpusReader
from nltk.text import Text

In [2]:
# nltk libraries
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HJfod\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [20]:
europarl_short_files = list(map(lambda p: os.path.basename(p), glob.glob('./europarl/en/ep-00-0*-*.txt')))
print(f"files in shortened corpus: {len(europarl_short_files)}")

files in shortened corpus: 48


In [19]:
europarl_read = EuroparlCorpusReader('./europarl/en', '.*')
europarl = Text(europarl_read.words())
europarl_short = Text(europarl_read.words(europarl_short_files))
print(f"length of whole corpus: {len(europarl)}, length of shortened corpus: {len(europarl_short)}")

length of whole corpus: 56968534, length of short corpus: 2675475


# now for the actual stuff

In [53]:
# https://stackoverflow.com/questions/49197667/nltk-how-to-get-bigrams-containing-a-specific-word
great_finder = TrigramCollocationFinder.from_documents([europarl_short])
great_finder.apply_ngram_filter(lambda *ngram: 'great' not in ngram)

large_finder = TrigramCollocationFinder.from_documents([europarl_short])
large_finder.apply_ngram_filter(lambda *ngram: 'large' not in ngram)

In [87]:
def print_table(header: str, list: list[(str, str, str)]):
    print(f'== {header} {"="*(33-4-len(header))}')
    for (a, b, c) in list:
        print(f'{a:<11} {b:<11} {c:<11}')
    print('')

In [90]:
bigram_measures = TrigramAssocMeasures()
print_table('uses of great', great_finder.nbest(bigram_measures.raw_freq, 15))
print_table('uses of large', large_finder.nbest(bigram_measures.raw_freq, 15))

== uses of great ================
a           great       deal       
great       deal        of         
is          a           great      
a           great       many       
of          the         great      
great       deal        to         
have        a           great      
great       importance  to         
is          of          great      
a           very        great      
be          a           great      
with        great       interest   
A           great       deal       
great       deal        more       
that        a           great      

== uses of large ================
large       number      of         
a           large       number     
to          a           large      
a           large       majority   
by          a           large      
a           very        large      
large       numbers     of         
large       majority    of         
large       part        of         
a           large       extent     
large       proportion  of     

In [92]:
europarl_short.concordance(['great', 'deal'])

Displaying 25 of 333 matches:
ent report, which I do not have a great deal to add to. I would like to take t
ustified. I would also ask that a great deal more emphasis be placed on these 
the Member States, we can avoid a great deal of problems later on. <P> Finally
 upon. Our motion will give you a great deal of food for thought, Commissioner
ntire European Parliament for the great deal of interest shown in competition-
s another area where Europe has a great deal in common, for example where the 
 the treaty texts submitted. So a great deal has to be done in this area. And 
means that, in any case, I have a great deal of respect for the briefing note 
 speaker, is that it is easier, a great deal easier, to offer help to victims 
. <P> In the meantime, there is a great deal of money and, Mr Gollnisch, I can
e Balkans and which has cost us a great deal more than we might have had to sp
with reform, none of which have a great deal of substance to them. So far, no 
ort - on which she has